In [1]:
import os
import re
import spacy
import pandas as pd
from collections import Counter

In [2]:
# Load Vietnamese NLP model
nlp = spacy.blank("vi")
nlp.add_pipe("sentencizer")

In [3]:
# Đọc file
file_path1 = "crawled/data_tuyen_sinh_phenikaa.txt"
file_path2 = "crawled/data_tuyen_sinh_ton_duc_thang.txt"

with open(file_path1, "r", encoding="utf-8") as file1:
    data1 = file1.read().strip().split("\n")

with open(file_path2, "r", encoding="utf-8") as file2:
    data2 = file2.read().strip().split("\n")

# Tạo DataFrame
df_phenikaa = pd.DataFrame(data1, columns=["phenikaa"])
df_tonducthang = pd.DataFrame(data2, columns=["tonducthang"])

# Kiểm tra dữ liệu
df_phenikaa.head(), df_tonducthang.head()

(                                            phenikaa
 0                             Đại học,Tin tuyển sinh
 1        Đại học (https://tuyensinh.edu.vn/dai-hoc/)
 2  Tin tuyển sinh (https://tuyensinh.edu.vn/tin-t...
 3  Trường Đại học Phenikaa công bố 5 phương thức ...
 4  Trường Đại học Phenikaa công bố thông tin tuyể...,
                                          tonducthang
 0                             Đại học,Tin tuyển sinh
 1        Đại học (https://tuyensinh.edu.vn/dai-hoc/)
 2  Tin tuyển sinh (https://tuyensinh.edu.vn/tin-t...
 3  Trường ĐH Tôn Đức Thắng chỉ dành 25% chỉ tiêu ...
 4  Theo phương án tuyển sinh dự kiến, năm 2022 Tr...)

In [4]:
def clean_text(text):
    """Xóa dấu câu (trừ . ! ?), giữ chữ thường"""
    text = re.sub(r"[^\w\s.!?]", "", text)  # Giữ lại dấu chấm, chấm than, chấm hỏi
    return text.strip()

df_phenikaa["phenikaa"] = df_phenikaa["phenikaa"].apply(clean_text)
df_tonducthang["tonducthang"] = df_tonducthang["tonducthang"].apply(clean_text)

# Kiểm tra dữ liệu sau làm sạch
df_phenikaa.head(), df_tonducthang.head()

(                                            phenikaa
 0                              Đại họcTin tuyển sinh
 1                Đại học httpstuyensinh.edu.vndaihoc
 2   Tin tuyển sinh httpstuyensinh.edu.vntintuyensinh
 3  Trường Đại học Phenikaa công bố 5 phương thức ...
 4  Trường Đại học Phenikaa công bố thông tin tuyể...,
                                          tonducthang
 0                              Đại họcTin tuyển sinh
 1                Đại học httpstuyensinh.edu.vndaihoc
 2   Tin tuyển sinh httpstuyensinh.edu.vntintuyensinh
 3  Trường ĐH Tôn Đức Thắng chỉ dành 25 chỉ tiêu x...
 4  Theo phương án tuyển sinh dự kiến năm 2022 Trư...)

In [5]:
def split_sentences(text):
    """Chia văn bản thành câu dựa trên dấu câu bằng spaCy"""
    doc = nlp(text)
    return [sent.text.strip() for sent in doc.sents if sent.text.strip()]

def count_words(text):
    """Đếm số từ bằng spaCy"""
    doc = nlp(text)
    return len([token for token in doc if token.is_alpha])

def count_stats(texts):
    """Đếm tổng số từ, câu và đoạn văn """
    total_words = sum(count_words(text) for text in texts)
    total_sentences = sum(len(split_sentences(text)) for text in texts)
    total_paragraphs = sum(len(text.strip().split("\n")) for text in texts)

    return {
        "Total Words": total_words,
        "Total Sentences": total_sentences,
        "Total Paragraphs": total_paragraphs
    }

# Thống kê cho từng tài liệu
stats_phenikaa = count_stats(df_phenikaa["phenikaa"])
stats_tonducthang = count_stats(df_tonducthang["tonducthang"])

stats_phenikaa, stats_tonducthang

({'Total Words': 816, 'Total Sentences': 153, 'Total Paragraphs': 79},
 {'Total Words': 285, 'Total Sentences': 57, 'Total Paragraphs': 27})

In [6]:
def compute_word_frequency(texts):
    """Tạo danh sách từ vựng ban đầu và tính tần suất"""
    vocab = Counter()
    for text in texts:
        words = text.split()
        vocab.update(words)
    return vocab

# Tính từ vựng ban đầu
vocab_phenikaa = compute_word_frequency(df_phenikaa["phenikaa"])
vocab_tonducthang = compute_word_frequency(df_tonducthang["tonducthang"])

# Hiển thị 10 từ phổ biến nhất
vocab_phenikaa.most_common(10), vocab_tonducthang.most_common(10)

([('tuyển', 62),
  ('học', 61),
  ('xét', 48),
  ('sinh', 40),
  ('các', 28),
  ('có', 23),
  ('điểm', 23),
  ('với', 22),
  ('từ', 21),
  ('Trường', 20)],
 [('tuyển', 21),
  ('học', 19),
  ('xét', 18),
  ('chỉ', 16),
  ('sinh', 14),
  ('ĐH', 13),
  ('trình', 12),
  ('THPT', 11),
  ('chương', 11),
  ('kết', 11)])

In [7]:
STOPWORDS_FILE = "vietnamese-stopwords.txt"
VIETNAMESE_STOPWORDS = set()

if os.path.exists(STOPWORDS_FILE):
    with open(STOPWORDS_FILE, "r", encoding="utf-8") as file:
        VIETNAMESE_STOPWORDS = set(file.read().strip().split("\n"))

def remove_stopwords(vocab):
    """Loại bỏ stopwords khỏi từ vựng"""
    return {word: freq for word, freq in vocab.items() if word not in VIETNAMESE_STOPWORDS}

# Tạo từ vựng mới sau khi loại bỏ stopwords
vocab_v2_phenikaa = remove_stopwords(vocab_phenikaa)
vocab_v2_tonducthang = remove_stopwords(vocab_tonducthang)

# Hiển thị 10 từ phổ biến nhất sau khi loại bỏ stopwords
sorted(vocab_v2_phenikaa.items(), key=lambda x: x[1], reverse=True)[:10], sorted(vocab_v2_tonducthang.items(), key=lambda x: x[1], reverse=True)[:10]


([('tuyển', 62),
  ('học', 61),
  ('xét', 48),
  ('sinh', 40),
  ('Trường', 20),
  ('hợp', 20),
  ('Đại', 18),
  ('thí', 17),
  ('môn', 15),
  ('trở', 15)],
 [('tuyển', 21),
  ('học', 19),
  ('xét', 18),
  ('sinh', 14),
  ('ĐH', 13),
  ('trình', 12),
  ('THPT', 11),
  ('chương', 11),
  ('kết', 11),
  ('Trường', 10)])

In [8]:
def extract_ngrams(texts):
    """Trích xuất unigrams, bigrams và trigrams"""
    unigrams, bigrams, trigrams = [], [], []
    
    for text in texts:
        words = text.split()
        unigrams.extend(words)
        bigrams.extend(zip(words[:-1], words[1:]))
        trigrams.extend(zip(words[:-2], words[1:-1], words[2:]))

    return unigrams, bigrams, trigrams

# Trích xuất n-grams
unigrams_p, bigrams_p, trigrams_p = extract_ngrams(df_phenikaa["phenikaa"])
unigrams_t, bigrams_t, trigrams_t = extract_ngrams(df_tonducthang["tonducthang"])

# Hiển thị kết quả
len(unigrams_p), len(bigrams_p), len(trigrams_p), len(unigrams_t), len(bigrams_t), len(trigrams_t)


(1816, 1737, 1662, 605, 578, 554)

In [12]:
def extract_anchor_texts(texts):
    """Trích xuất anchor text từ danh sách văn bản có dạng text (link)"""
    anchor_texts = []
    
    # Regex tìm text có dạng "text (URL)"
    pattern = r"(\S.*?)(?:\s*)\((https?://[^\s)]+)\)"

    # Chuyển đổi pandas.Series thành danh sách chuỗi
    if isinstance(texts, pd.Series):
        texts = texts.dropna().astype(str).tolist()

    # Duyệt từng chuỗi trong danh sách
    for text in texts:
        matches = re.findall(pattern, text)
        anchor_texts.extend([match[0].strip() for match in matches])

    return anchor_texts

# Trích xuất anchor text
df_phenikaa["phenikaa"] = df_phenikaa["phenikaa"].apply(lambda x: extract_anchor_texts([x]) if pd.notna(x) else [])
anchors_t = extract_anchor_texts(df_tonducthang["tonducthang"])

print("Phenikaa:", df_phenikaa["phenikaa"])
print("Ton Duc Thang:", anchors_t)


Phenikaa: 0     []
1     []
2     []
3     []
4     []
      ..
74    []
75    []
76    []
77    []
78    []
Name: phenikaa, Length: 79, dtype: object
Ton Duc Thang: []


C:\Users\linh.nguyenkhanh3\AppData\Local\Temp\ipykernel_7112\1058200553.py:20: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  df_phenikaa["phenikaa"] = df_phenikaa["phenikaa"].apply(lambda x: extract_anchor_texts([x]) if pd.notna(x) else [])


In [10]:
def extract_named_entities(texts):
    """Thực hiện Named Entity Recognition (NER)"""
    entities = []
    for text in texts:
        doc = nlp(text)
        for ent in doc.ents:
            entities.append((ent.text, ent.label_))
    return entities

# Thực hiện NER
ner_phenikaa = extract_named_entities(df_phenikaa["phenikaa"])
ner_tonducthang = extract_named_entities(df_tonducthang["tonducthang"])

ner_phenikaa[:10], ner_tonducthang[:10]

([], [])